In [318]:
import matplotlib.pyplot as plt
import cv2 as cv
import numpy as np

image_count = 6*8
X = np.zeros((image_count, 112*92))
for i in range(6):
    for j in range(8):
        image = cv.imread(f'images/s{i+1}/{j+1}.pgm', cv.IMREAD_GRAYSCALE)
        image_arr = np.resize(image, (112*92))
        X[8*i+j] = image_arr

mean_face = np.mean(X, axis=0)
for i in range(image_count):
    X[i] -= mean_face

In [319]:
# Transpose to get the correct dimensions
U, S, Vh = np.linalg.svd(X.transpose())

In [ ]:
k = 20
U_k = U.copy()
U_k[:, k+1:] = 0

In [324]:
test_image = cv.imread(f'images/s5/10.pgm', cv.IMREAD_GRAYSCALE)
test_image_arr = np.resize(test_image, (112*92))
test_image_coord = U_k.transpose() @ (test_image_arr - mean_face)

best_idx = -1
best_fit = np.inf
for i in range(image_count):
    image_arr = X[i]
    image_coord = U_k.transpose() @ image_arr
    fit = np.linalg.norm(image_coord - test_image_coord)

    if fit < best_fit:
        best_idx = i
        best_fit = fit

print(f"Image is from s{int(best_idx/8)+1}")

Image is from s5
